In [72]:
# read data
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_excel('/Users/kuzaowuwei/Desktop/2022美赛/赛中数据/2决策模型/data/price_volatility.xlsx')
df2 = pd.read_excel('/Users/kuzaowuwei/Desktop/2022美赛/赛中数据/2决策模型/data/price_pred.xlsx')
R_g = df['u_gold']
R_b = df['u_bitcoin']
sigma_g = df['sigma_gold'] 
sigma_b = df['sigma_bitcoin']
pred_g = df2['gold_pred']
pred_b = df2['bitcoin_pred']

In [63]:
from sko.PSO import PSO
import numpy as np

penalty_rate = -4
T = 4
def demo_func(x):
    # c = x[:1000]
    g = x[:T]
    b = x[T:]
    C = 1000
    G = 0
    B = 0
    TR = 0
    for t in range(T):
        tran_loss =  abs(g[t]) * 0.01 + abs(b[t]) * 0.02
        c = -tran_loss-b[t]-g[t]
        psnt_return = c + (G + g[t]) * (1+R_g[t]) + (B + b[t]) * (1+R_b[t]) - G - B
        C = C+c
        G = (G + g[t]) * (1+R_g[t])
        B = (B + b[t]) * (1+R_b[t])
        TR += psnt_return * 1024
        if C < 0:
            # TR += np.exp(-C)
            TR += ((C)**2 + 1) * penalty_rate * t
        if G < 0:
            # TR += np.exp(-G)

            TR += ((G)**2 +1)*penalty_rate  * t
        if B < 0:
            # TR += np.exp(-B)

            TR += ((B)**2 + 1) *penalty_rate * t
    
    return -TR

pso_lb = [-500]*T*2
pso_lb[0] = 0
pso_lb[T] = 0
pso_ub = [500]*T*2

pso = PSO(func=demo_func, dim=2*T,max_iter = 10000,lb= pso_lb,ub= pso_ub,pop=10)
fitness = pso.run()
# print('best_x is ',pso.gbest_x)
print('best_y is ',-pso.gbest_y)
# print(fitness)
# pso.plot_history()
# pso.
for i in range(T):
    print(pso.gbest_x[i], pso.gbest_x[i+T])

best_y is  [-11335.40959196]
55.762076688901956 0.0
-4.139952991917921 5.173768757381703e-05
500.0 0.056362864063600485
-500.0 -0.022530962377304693


In [79]:
import numpy as np
import pandas as pd
df = pd.read_excel('/Users/kuzaowuwei/Desktop/2022美赛/赛中数据/2决策模型/data/price_volatility.xlsx')
R_g = df['u_gold']
R_b = df['u_bitcoin']
# print(R_g.shape)
def cal_return(g,b,a1=0.01,a2=0.02,c0=1000):
    # g,b -> (1826,)
    n = len(g)
    C = np.zeros(n)
    G = np.zeros(n)
    B = np.zeros(n)
    C[0] = 1000
    portfolio_value = np.zeros(n)
    # TR = 0
    for t in range(n-1):
        tran_loss =  abs(g[t]) * a1 + abs(b[t]) * a2
        c = -tran_loss-b[t]-g[t]
        # print(C[t])
        C[t] = C[t] + c
        G[t] = (G[t] + g[t])
        B[t] = (B[t] + b[t])
        portfolio_value[t] = C[t] + G[t] + B[t]
        print(t,C[t],G[t],B[t],portfolio_value[t],sep='\t')
        C[t+1] = C[t]
        G[t+1] = G[t] * (1+R_g[t])
        B[t+1] = B[t] * (1+R_b[t])
    
    portfolio_value[n-1] = C[n-1] + G[n-1] + B[n-1]
    TR = portfolio_value[n-1] - portfolio_value[0]
    return TR,C,G,B,portfolio_value

# g = np.zeros(1826)
# b = np.zeros(1826)
# g = pso.gbest_x[:T]
# b = pso.gbest_x[T:]
# TR,C,G,B,portfolio_value = cal_return(g,b)

In [6]:
TR

0.0

In [80]:
def simulate_resolution():
    n = len(R_g)
    a1 = 0.01
    a2 = 0.02
    c = np.zeros(n)
    g = np.zeros(n)
    b = np.zeros(n)
    C = np.zeros(n)
    G = np.zeros(n)
    B = np.zeros(n)
    C[0] = 1000
    portfolio_value = np.zeros(n)
    r1 = 0.25
    r2 = 0.5
    for t in range(n-1):
        if abs(R_g[t+1]) > a1 :
            if R_g[t+1] < 0:
                g[t] = -G[t]*r2
            else:
                g[t] = C[t]*r1

        if abs(R_b[t+1])>a2:
            if R_b[t+1]<0:
                b[t] = -B[t]*r2
            else:
                b[t] = C[t]*r1 
        tran_loss = abs(g[t]) * a1 + abs(b[t]) * a2
        c = -tran_loss-b[t]-g[t]
        # print(C[t])
        C[t] = C[t] + c
        G[t] = (G[t] + g[t])
        B[t] = (B[t] + b[t])

        C[t+1] = C[t]
        G[t+1] = G[t] * (1+R_g[t])
        B[t+1] = B[t] * (1+R_b[t])
    return g,b

g,b = simulate_resolution()
TR,C,G,B,portfolio_value = cal_return(g,b)

0	1000.0	0.0	0.0	1000.0
1	1000.0	0.0	0.0	1000.0
2	1000.0	0.0	0.0	1000.0
3	1000.0	0.0	0.0	1000.0
4	1000.0	0.0	0.0	1000.0
5	1000.0	0.0	0.0	1000.0
6	1000.0	0.0	0.0	1000.0
7	1000.0	0.0	0.0	1000.0
8	1000.0	0.0	0.0	1000.0
9	1000.0	0.0	0.0	1000.0
10	1000.0	0.0	0.0	1000.0
11	1000.0	0.0	0.0	1000.0
12	1000.0	0.0	0.0	1000.0
13	1000.0	0.0	0.0	1000.0
14	1000.0	0.0	0.0	1000.0
15	1000.0	0.0	0.0	1000.0
16	1000.0	0.0	0.0	1000.0
17	1000.0	0.0	0.0	1000.0
18	1000.0	0.0	0.0	1000.0
19	1000.0	0.0	0.0	1000.0
20	1000.0	0.0	0.0	1000.0
21	1000.0	0.0	0.0	1000.0
22	1000.0	0.0	0.0	1000.0
23	1000.0	0.0	0.0	1000.0
24	1000.0	0.0	0.0	1000.0
25	1000.0	0.0	0.0	1000.0
26	1000.0	0.0	0.0	1000.0
27	1000.0	0.0	0.0	1000.0
28	1000.0	0.0	0.0	1000.0
29	1000.0	0.0	0.0	1000.0
30	745.0	0.0	250.0	995.0
31	745.0	0.0	248.87863191264879	993.8786319126488
32	745.0	0.0	257.0724294620112	1002.0724294620112
33	745.0	0.0	257.4570248893655	1002.4570248893656
34	745.0	0.0	256.67164964629086	1001.6716496462909
35	745.0	0.0	258.25858452080627	10

In [81]:
df_res = pd.DataFrame()
df_res['date'] = df['date']
df_res['delta_g'] = g
df_res['delta_b'] = b
df_res['C'] = C
df_res['G'] = G
df_res['B'] = B
df_res['value'] = portfolio_value
df_res.to_excel('/Users/kuzaowuwei/Desktop/2022美赛/赛中数据/2决策模型/data/resolution_1.xlsx')
